In [1]:

#  Setting display options 

import pandas as pd
   #
pd.set_option("display.width", 480)

#  Sets horizontal scroll for wide outputs
#
from IPython.display import display, HTML
display(HTML(""))

from tabulate import tabulate

print("--")


--


#  Setup stuff: Connectivity

In [2]:

from katana import remote
from katana.remote import import_data

my_client = remote.Client()

print(my_client)


In [3]:

NUM_PARTITIONS  = 3
   #
DB_NAME         = "my_db"
GRAPH_NAME      = "my_graph"

print("--")


--


In [4]:

#  CONNECT TO GRAPH

my_graph, *_ = my_client.get_database(name=DB_NAME).find_graphs_by_name(GRAPH_NAME)

print(my_graph)


<_Graph my_graph, AMrqQvvsEEdXVLM72uaoBZgQBD5abQbFKeG7QsPFAdgY, 0>


# Louvain algorithm

In [5]:

from katana.remote import analytics


l_cntr = 20                                                #  Used to generate a unique column name
   #
print("--")


--


In [6]:

#  We use this counter to generate a unique column name below 
#
l_cntr += 1
   #
l_output_propname     = "louvain_" + str(l_cntr).zfill(4)


analytics.louvain_clustering(
   my_graph,
      #
   result_property_name = l_output_propname,
      #
   is_symmetric         = True
   )

print("--")


          0/? [?op/s]

--


# Checking that result above


In [8]:

#  Unfiltered node data
#

l_query = """

   MATCH (n)
   RETURN n.{0}, ID(n), n
   ORDER BY n.{0} DESC
    
    """.format(l_output_propname)

l_result = my_graph.query_unpaginated(l_query)
   #
print(tabulate(l_result, headers='keys', tablefmt='psql'))

#  Sample data
#
#     +----+----------------------+-----------------+-----------------+-------------+-----------+------------------+------------------+----------------------+----------+
#     |    |       n.louvain_0021 |           ID(n) |   n.internal_id | n.labels    | n.LABEL   | n.airport_code   | n.airport_name   |       n.louvain_0021 | n.type   |
#     |----+----------------------+-----------------+-----------------+-------------+-----------+------------------+------------------+----------------------+----------|
#     |  0 | 18446744073709551615 |               0 |               0 | ['Airport'] | Airport   | SJC              | San Jose         | 18446744073709551615 | node     |
#     |  1 | 18446744073709551615 | 281474976710656 | 281474976710656 | ['Airport'] | Airport   | MKE              | Milwaukee        | 18446744073709551615 | node     |
#     |  2 | 18446744073709551615 | 562949953421312 | 562949953421312 | ['Airport'] | Airport   | ORD              | Chicago O-Hare   | 18446744073709551615 | node     |
#     |  3 | 18446744073709551615 | 562949953421313 | 562949953421313 | ['Airport'] | Airport   | DEN              | Denver           | 18446744073709551615 | node     |
#     +----+----------------------+-----------------+-----------------+-------------+-----------+------------------+------------------+----------------------+----------+

#  Sample data, after adding Narita and Seoul below-
#
#     +----+----------------------+-----------------+-----------------+-------------+------------------+------------------+----------------------+----------+-----------+
#     |    |       n.louvain_0023 |           ID(n) |   n.internal_id | n.labels    | n.airport_code   | n.airport_name   |       n.louvain_0023 | n.type   | n.LABEL   |
#     |----+----------------------+-----------------+-----------------+-------------+------------------+------------------+----------------------+----------+-----------|
#     |  0 | 18446744073709551615 |               2 |               2 | ['Airport'] | NRT              | Narita, Japan    | 18446744073709551615 | node     | nan       |
#     |  1 | 18446744073709551615 |               0 |               0 | ['Airport'] | ORD              | Chicago O-Hare   | 18446744073709551615 | node     | Airport   |
#     |  2 | 18446744073709551615 |               1 |               1 | ['Airport'] | SJC              | San Jose         | 18446744073709551615 | node     | Airport   |
#     |  3 | 18446744073709551615 | 281474976710656 | 281474976710656 | ['Airport'] | ICN              | Seoul, Korea     | 18446744073709551615 | node     | nan       |
#     |  4 | 18446744073709551615 | 562949953421312 | 562949953421312 | ['Airport'] | MKE              | Milwaukee        | 18446744073709551615 | node     | Airport   |
#     |  5 | 18446744073709551615 | 562949953421313 | 562949953421313 | ['Airport'] | DEN              | Denver           | 18446744073709551615 | node     | Airport   |
#     +----+----------------------+-----------------+-----------------+-------------+------------------+------------------+----------------------+----------+-----------+


          0/? [?op/s]

/opt/conda/lib/python3.8/site-packages/katana/remote/result_set.py:69: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  self._response = response


In [15]:

#  View output
#

l_query = """

   MATCH (n)
   RETURN n.{0}, ID(n), n.airport_name
   ORDER BY n.{0} DESC
   LIMIT 10
    
    """.format(l_output_propname)

l_result = my_graph.query(l_query)
   #    
print(tabulate(l_result, headers='keys', tablefmt='psql'))


          0/? [?op/s]

+----+----------------------+-----------------+------------------+
|    |       n.louvain_0023 |           ID(n) | n.airport_name   |
|----+----------------------+-----------------+------------------|
|  0 | 18446744073709551615 |               0 | Chicago O-Hare   |
|  1 | 18446744073709551615 |               1 | San Jose         |
|  2 | 18446744073709551615 |               2 | Narita, Japan    |
|  3 | 18446744073709551615 | 281474976710656 | Seoul, Korea     |
|  4 | 18446744073709551615 | 562949953421312 | Milwaukee        |
|  5 | 18446744073709551615 | 562949953421313 | Denver           |
+----+----------------------+-----------------+------------------+


In [12]:

#  Add a new community
#

l_query = """
   CREATE ( n: Airport { airport_code: 'NRT' } )               //  Notice single curly braces
   SET n.airport_name = 'Narita, Japan' 
   """
      #
display(l_result = my_graph.query(l_query))

l_query = """
   CREATE ( n: Airport { airport_code: 'ICN' } ) 
   SET n.airport_name = 'Seoul, Korea' 
   """
      #
display(l_result = my_graph.query(l_query))


l_query = """
   MATCH
      (n: Airport   ),
      (m: Airport   )
   WHERE n.airport_code     = 'NRT' AND m.airport_code  = 'ICN'
   CREATE (n) -[r: FLIES_TO { DISTANCE: 752, NUM_HOPS: 1}]-> (m)
   """
      #
display(l_result = my_graph.query(l_query))



          0/? [?op/s]

          0/? [?op/s]

          0/? [?op/s]

In [ ]:
#  Go up, run Louvain again ..

#  To remove a given property name from all nodes

In [ ]:

#  It is only required that you run this one time.
#
#  You may change the value i nthe text entry field as often as you wish.
#
from ipywidgets import widgets

my_column  = widgets.Text(
    value        = "bt_0020",
    placeholder  = "type here",
    description  = "Column Name: ",
    disabled     = False
)

my_column


In [ ]:

#  We could run the remove_property() directly, but then we couldn't cleanly catch
#  errors. So, run it inside a UDF.
#

def f_drop_column(i_graph, i_property):
   from katana import distributed
   try:
      i_graph.nodes.remove_property(i_property)
      return distributed.single_host(host=0, result=True)
   except LookupError as e:
      return distributed.single_host(host=0, result=False)
   except Exception as e:
      return distributed.single_host(host=0, result=False)
        

print("Deleting column name: %s" % (my_column.value))
   #
l_column = str(my_column.value)


if (my_graph.run(lambda g: f_drop_column(g, l_column))):
   print("Column ( %s ) deleted." % (my_column.value))
else:
   print("Column ( %s ) not found." % (my_column.value))
